# Persian Text Classification [DigiMag, Persian News]

The task target is labeling texts in a supervised manner in both existing datasets **DigiMag** and **Persian News**.

In [1]:
!nvidia-smi
!lscpu

Sat Jul 31 12:23:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install hazm==0.7.0
!pip install seqeval==1.2.2
!pip install sentencepiece==0.1.96
!pip install transformers==4.7.0
!pip install clean-text[gpl]==0.4.0

     |████████████████████████████████| 316 kB 8.4 MB/s 
     |████████████████████████████████| 233 kB 60.4 MB/s 
     |████████████████████████████████| 1.4 MB 53.0 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394485 sha256=df4c73fa13275023a5401eb1ef83450d0ab661ab6cd050e67be3c403cd9fe896
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154549 sha256=c18527d12175d77f8f78f5af31c5108d4a0f9063dc1a9c192e3fc5849284410e
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 43 kB 1.3 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.

In [3]:
!pip install PyDrive
import os
import IPython.display as ipd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# Import required packages
import os
import gc
import re
import hazm
import time
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import transformers
from transformers import AutoConfig, AutoTokenizer
from transformers import AutoModelForSequenceClassification

from cleantext import clean

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print()
print('numpy', np.__version__)
print('pandas', pd.__version__)
print('transformers', transformers.__version__)
print('torch', torch.__version__)
print()

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



numpy 1.19.5
pandas 1.1.5
transformers 4.7.0
torch 1.9.0+cu102

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
class TextClassificationDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Text Classification. """

    def __init__(self, tokenizer, comments, targets, label_list=None, max_len=128):
        self.comments = comments
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label2index = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}
        self.index2label = {i: label for label, i in self.label2index.items()}

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, item):
        comment = self.comments[item]
        target = self.label2index[self.targets[item]]
        encoding = self.tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt'
        )

        inputs = {
            'comment': comment,
            'targets': torch.tensor(target, dtype=torch.long),
            'original_targets': self.targets[item],
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        return inputs


class TextClassifier:
    def __init__(self, model_name):
        self.normalizer = hazm.Normalizer()
        self.model_name = model_name
        self.config = AutoConfig.from_pretrained(self.model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name)
        self.id2label = self.config.id2label
        self.label2id = self.config.label2id

    def cleaning(self, text):
        def cleanhtml(raw_html):
            clean_pattern = re.compile('<.*?>')
            clean_text = re.sub(clean_pattern, '', raw_html)
            return clean_text

        if type(text) is not str:
            return None

        text = text.strip()

        # regular cleaning
        text = clean(
            text,
            fix_unicode=True,
            to_ascii=False,
            lower=True,
            no_line_breaks=True,
            no_urls=True,
            no_emails=True,
            no_phone_numbers=True,
            no_numbers=False,
            no_digits=False,
            no_currency_symbols=True,
            no_punct=False,
            replace_with_url="",
            replace_with_email="",
            replace_with_phone_number="",
            replace_with_number="",
            replace_with_digit="0",
            replace_with_currency_symbol=""
        )

        # cleaning htmls
        text = cleanhtml(text)

        # normalizing
        text = self.normalizer.normalize(text)

        # removing wierd patterns
        wierd_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001f926-\U0001f937"
                                   u'\U00010000-\U0010ffff'
                                   u"\u200d"
                                   u"\u2640-\u2642"
                                   u"\u2600-\u2B55"
                                   u"\u23cf"
                                   u"\u23e9"
                                   u"\u231a"
                                   u"\u3030"
                                   u"\ufe0f"
                                   u"\u2069"
                                   u"\u2066"
                                   # u"\u200c"
                                   u"\u2068"
                                   u"\u2067"
                                   "]+", flags=re.UNICODE)

        text = wierd_pattern.sub(r'', text)

        # removing extra spaces, hashtags
        text = re.sub("#", "", text)
        text = re.sub("\s+", " ", text)
        if text in ['', " "]:
            return None
        return text

    def load_dataset_test_file(self, dataset_name, dataset_file, **kwargs):
        if dataset_name.lower() == "digimag":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            data = pd.read_csv(dataset_file, delimiter="\t")
            # drop label_id because its not consistent with albert model labels!
            data = data[['content', 'label']]

            # cleaning comments
            data = data.dropna(subset=['content'])
            data['content'] = data['content'].apply(self.cleaning)
            data = data.dropna(subset=['content'])

            data['label_id'] = data['label'].apply(lambda t: self.label2id[t])
            x_test, y_test = data['content'].values.tolist(), data['label_id'].values.tolist()
            print(f'test part:\n #content: {len(x_test)}, #labels: {len(y_test)}')
            return x_test, y_test
        if dataset_name.lower() == "persian-news":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            data = pd.read_csv(dataset_file, delimiter="\t")
            # drop label_id because its not consistent with albert model labels!
            data = data[['content', 'label']]

            # cleaning comments
            data = data.dropna(subset=['content'])
            data['content'] = data['content'].apply(self.cleaning)
            data = data.dropna(subset=['content'])

            data['label_id'] = data['label'].apply(lambda t: self.label2id[t])
            x_test, y_test = data['content'].values.tolist(), data['label_id'].values.tolist()
            print(f'test part:\n #content: {len(x_test)}, #labels: {len(y_test)}')
            return x_test, y_test

    def text_classification_inference(self, input_text):
        if not self.model or not self.tokenizer or not self.id2label:
            print('Something wrong has been happened!')
            return

        pt_batch = self.tokenizer(
            input_text,
            padding=True,
            truncation=True,
            max_length=self.config.max_position_embeddings,
            return_tensors="pt"
        )

        pt_outputs = self.model(**pt_batch)
        pt_predictions = torch.argmax(F.softmax(pt_outputs.logits, dim=1), dim=1)

        output_predictions = []
        for i, sentence in enumerate(input_text):
            output_predictions.append((sentence, self.id2label.get(pt_predictions[i].item())))
        return output_predictions

    def evaluation(self, input_text, input_labels, device, batch_size=4):
        if not self.model or not self.tokenizer or not self.id2label:
            print('Something wrong has been happened!')
            return

        max_len = self.config.max_position_embeddings
        label_list = list(set(input_labels))
        label_count = {self.id2label[label]: input_labels.count(label) for label in label_list}
        print("label_count:", label_count)
        dataset = TextClassificationDataset(comments=input_text, targets=input_labels, tokenizer=self.tokenizer,
                                            max_len=max_len, label_list=label_list)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

        print("#samples:", len(input_text))
        print("#batch:", len(data_loader))

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()

        total_loss, total_time = 0, 0
        output_predictions = []
        golden_labels, predicted_labels = [], []
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            b_comments = batch['comment']
            b_input_ids = batch['input_ids']
            b_attention_mask = batch['attention_mask']
            b_token_type_ids = batch['token_type_ids']
            b_targets = batch['targets']

            # move tensors to GPU if CUDA is available
            b_input_ids = b_input_ids.to(device)
            b_attention_mask = b_attention_mask.to(device)
            b_token_type_ids = b_token_type_ids.to(device)
            b_targets = b_targets.to(device)

            # This will return the loss (rather than the model output) because we have provided the `labels`.
            with torch.no_grad():
                start = time.monotonic()
                b_outputs = self.model(input_ids=b_input_ids, attention_mask=b_attention_mask,
                                       token_type_ids=b_token_type_ids, labels=b_targets)
                end = time.monotonic()
                total_time += end - start
                print(f'inference time for step {step}: {end - start}')
            # get the loss
            total_loss += b_outputs.loss.item()

            b_original_targets = batch['original_targets']
            golden_labels.extend(b_original_targets.tolist())

            b_predictions = torch.argmax(F.softmax(b_outputs.logits, dim=1), dim=1)
            b_predictions = b_predictions.cpu().detach().numpy().tolist()
            b_predictions = [dataset.index2label[label] for label in b_predictions]
            predicted_labels.extend(b_predictions)

            for i, comment in enumerate(b_comments):
                output_predictions.append((
                    comment,
                    self.id2label[b_original_targets[i].item()],
                    self.id2label[b_predictions[i]]
                ))
                # print(f'output prediction: {i},{comment},{self.id2label[b_original_targets[i].item()]},'
                #       f'{self.id2label[b_predictions[i]]}')

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(data_loader)
        print("average loss:", avg_train_loss)
        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(input_text))

        # evaluate
        print("Test Accuracy: {}".format(accuracy_score(golden_labels, predicted_labels)))
        print("Test Precision: {}".format(precision_score(golden_labels, predicted_labels, average="weighted")))
        print("Test Recall: {}".format(recall_score(golden_labels, predicted_labels, average="weighted")))
        print("Test F1-Score(weighted average): {}".format(
            f1_score(golden_labels, predicted_labels, average="weighted")))
        print("Test classification Report:\n{}".format(classification_report(
            golden_labels, predicted_labels, digits=10, target_names=[self.id2label[_] for _ in sorted(label_list)])))
        return output_predictions


In [6]:
model_name='HooshvareLab/bert-fa-base-uncased-clf-digimag'
tc_model = TextClassifier(model_name)
print(tc_model.config)


BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "digimag",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "\u0628\u0627\u0632\u06cc \u0648\u06cc\u062f\u06cc\u0648\u06cc\u06cc",
    "1": "\u0631\u0627\u0647\u0646\u0645\u0627\u06cc \u062e\u0631\u06cc\u062f",
    "2": "\u0633\u0644\u0627\u0645\u062a \u0648 \u0632\u06cc\u0628\u0627\u06cc\u06cc",
    "3": "\u0639\u0644\u0645 \u0648 \u062a\u06a9\u0646\u0648\u0644\u0648\u0698\u06cc",
    "4": "\u0639\u0645\u0648\u0645\u06cc",
    "5": "\u0647\u0646\u0631 \u0648 \u0633\u06cc\u0646\u0645\u0627",
    "6": "\u06a9\u062a\u0627\u0628 \u0648 \u0627\u062f\u0628\u06cc\u0627\u062a"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "\u0628\u0627\u0632\u06cc \u0648\u06cc\u062f\u06cc\u0648\u06cc\u06cc": 0,
    "\u0631\u0627\u0647\u0646\u0645\u0627\u

In [7]:
print(tc_model.id2label)
print(tc_model.label2id)

{0: 'بازی ویدیویی', 1: 'راهنمای خرید', 2: 'سلامت و زیبایی', 3: 'علم و تکنولوژی', 4: 'عمومی', 5: 'هنر و سینما', 6: 'کتاب و ادبیات'}
{'بازی ویدیویی': 0, 'راهنمای خرید': 1, 'سلامت و زیبایی': 2, 'علم و تکنولوژی': 3, 'عمومی': 4, 'هنر و سینما': 5, 'کتاب و ادبیات': 6}


## DigiMag
A total of 8,515 articles scraped from Digikala Online Magazine. This dataset includes seven different classes.

|          Label         | # | 
|:------------------------:|:-----------:|
|  Video Games  |      1967    |
|  Shopping Guide |      125      |
|  Health Beauty |      1610      |
|  Science Technology |      2772      |
|  General |      120      |
|  Art Cinema |      1667      |
|  Books Literature |      254      |

Download You can download the dataset from [here](https://drive.google.com/uc?id=1YgrCYY-Z0h2z0-PfWVfOGt1Tv0JDI-qz).

In [8]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download = drive.CreateFile({'id': '1YgrCYY-Z0h2z0-PfWVfOGt1Tv0JDI-qz'})
download.GetContentFile('digimag.zip')
!ls

adc.json  digimag.zip  sample_data


In [9]:
!unzip digimag.zip
!ls
!ls digimag

Archive:  digimag.zip
   creating: digimag/
  inflating: digimag/dev.csv         
  inflating: digimag/train.csv       
  inflating: digimag/test.csv        
adc.json  digimag  digimag.zip	sample_data
dev.csv  test.csv  train.csv


In [10]:
test_comments, test_labels = tc_model.load_dataset_test_file(dataset_name="digimag", dataset_file="./digimag/test.csv")
print(test_comments[:5])
print(test_labels[:5])
print(len(test_comments))
print(len(test_labels))

test part:
 #content: 852, #labels: 852
['با این ۵ صبحانه رژیمی لاغر شوید صبحانه همیشه به عنوان مهم\u200cترین وعده غذایی در طول روز شناخته می\u200cشود چرا که انرژی مورد نیاز بدن بعد از بیدار شدن از خواب را تامین می\u200cکند. با این وجود آیا می\u200cدانستید که می\u200cتوانید مصرف این وعده را به سمتی پیش ببرید که با مصرف صبحانه رژیمی وزن کم کنید و لاغر شوید؟ در واقع با توجه به مواد موجود در این وعده حیاتی، صبحانه به یک عامل مهم برای لاغری تبدیل می\u200cشود. زمانی که رژیم غذایی متعادلی را مصرف می\u200cکنید و چربی سوزی دارید، اولین وعده\u200cای که بعد از مدت طولانی غذا نخوردن می\u200cخورید، اهمیت زیادی پیدا می\u200cکند. بنابراین می\u200cتوان مواد غذایی مناسبی را با توجه به این نکته انتخاب کرد. در وعده صبحانه بهتر است ۲۵-۳۵ درصد از میزان کالری که در طول روز سوزانده می\u200cشود را مصرف کنید. با این وجود یک صبحانه متعادل حاوی ۴۰۰-۴۵۰ کالری از ۲۰۰۰ کالری است که در طول روز مصرف می\u200cشود. بیشتر بخوانید: بهترین غذاهایی که می\u200cتوان برای صبحانه خورد کدامند؟ تحقیقی که در دانشگاه میسوری توسط د

In [11]:
tc_model.text_classification_inference(test_comments[:5])

[('با این ۵ صبحانه رژیمی لاغر شوید صبحانه همیشه به عنوان مهم\u200cترین وعده غذایی در طول روز شناخته می\u200cشود چرا که انرژی مورد نیاز بدن بعد از بیدار شدن از خواب را تامین می\u200cکند. با این وجود آیا می\u200cدانستید که می\u200cتوانید مصرف این وعده را به سمتی پیش ببرید که با مصرف صبحانه رژیمی وزن کم کنید و لاغر شوید؟ در واقع با توجه به مواد موجود در این وعده حیاتی، صبحانه به یک عامل مهم برای لاغری تبدیل می\u200cشود. زمانی که رژیم غذایی متعادلی را مصرف می\u200cکنید و چربی سوزی دارید، اولین وعده\u200cای که بعد از مدت طولانی غذا نخوردن می\u200cخورید، اهمیت زیادی پیدا می\u200cکند. بنابراین می\u200cتوان مواد غذایی مناسبی را با توجه به این نکته انتخاب کرد. در وعده صبحانه بهتر است ۲۵-۳۵ درصد از میزان کالری که در طول روز سوزانده می\u200cشود را مصرف کنید. با این وجود یک صبحانه متعادل حاوی ۴۰۰-۴۵۰ کالری از ۲۰۰۰ کالری است که در طول روز مصرف می\u200cشود. بیشتر بخوانید: بهترین غذاهایی که می\u200cتوان برای صبحانه خورد کدامند؟ تحقیقی که در دانشگاه میسوری توسط دانشمندان انجام شد، نشان می\u200cدهد مصر

In [12]:
!nvidia-smi
!lscpu

Sat Jul 31 12:28:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
evaluation_output = tc_model.evaluation(test_comments, test_labels, device, batch_size=128)

label_count: {'بازی ویدیویی': 197, 'راهنمای خرید': 13, 'سلامت و زیبایی': 161, 'علم و تکنولوژی': 277, 'عمومی': 12, 'هنر و سینما': 167, 'کتاب و ادبیات': 25}
#samples: 852
#batch: 7
Start to evaluate test data ...
inference time for step 0: 0.13760050199999796
inference time for step 1: 0.013303419999999733
inference time for step 2: 0.00837771999999859
inference time for step 3: 0.008319506000020738
inference time for step 4: 0.009523607999994965
inference time for step 5: 0.009177677999957723
inference time for step 6: 0.009498633000021073
average loss: 0.18178633547254971
total inference time: 0.19580106699999078
total inference time / #samples: 0.00022981345892017697
Test Accuracy: 0.9565727699530516
Test Precision: 0.9531710556126156
Test Recall: 0.9565727699530516
Test F1-Score(weighted average): 0.9536547265563312
Test classification Report:
                precision    recall  f1-score   support

  بازی ویدیویی  0.9897435897 0.9796954315 0.9846938776       197
  راهنمای خرید  0.77

In [14]:
for comment, true_label, predicted_label in evaluation_output[:25]:
  print('{}\t{}\t{}'.format(comment, true_label, predicted_label))

با این ۵ صبحانه رژیمی لاغر شوید صبحانه همیشه به عنوان مهم‌ترین وعده غذایی در طول روز شناخته می‌شود چرا که انرژی مورد نیاز بدن بعد از بیدار شدن از خواب را تامین می‌کند. با این وجود آیا می‌دانستید که می‌توانید مصرف این وعده را به سمتی پیش ببرید که با مصرف صبحانه رژیمی وزن کم کنید و لاغر شوید؟ در واقع با توجه به مواد موجود در این وعده حیاتی، صبحانه به یک عامل مهم برای لاغری تبدیل می‌شود. زمانی که رژیم غذایی متعادلی را مصرف می‌کنید و چربی سوزی دارید، اولین وعده‌ای که بعد از مدت طولانی غذا نخوردن می‌خورید، اهمیت زیادی پیدا می‌کند. بنابراین می‌توان مواد غذایی مناسبی را با توجه به این نکته انتخاب کرد. در وعده صبحانه بهتر است ۲۵-۳۵ درصد از میزان کالری که در طول روز سوزانده می‌شود را مصرف کنید. با این وجود یک صبحانه متعادل حاوی ۴۰۰-۴۵۰ کالری از ۲۰۰۰ کالری است که در طول روز مصرف می‌شود. بیشتر بخوانید: بهترین غذاهایی که می‌توان برای صبحانه خورد کدامند؟ تحقیقی که در دانشگاه میسوری توسط دانشمندان انجام شد، نشان می‌دهد مصرف صبحانه هورمون گرلین و برخی مواد شیمیایی در مغز که با اشتها در ارتباط هستند ر

In [15]:
output_file_name = "text_classification_digimag_testset_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for comment, true_label, predicted_label in evaluation_output:
    output_file.write('{}\t{}\t{}\n'.format(comment, true_label, predicted_label))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()